In [98]:
import pandas as pd
import numpy as np
import random
import json
import matplotlib.pyplot as plt
import pickle


from gtts import gTTS
from io import BytesIO
import pygame
import speech_recognition as sr

import tflearn
import tensorflow as tf
import nltk
from nltk.stem.lancaster import LancasterStemmer


+ Reading Json file of previously created intents and storing them in diffferent formats for ease of access

In [93]:
with open('intents.json') as file:
    data=json.load(file)
data['intents']


[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up',
   'good morning',
   'good afternoon'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day',
   'bye',
   'good night'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'have a great day'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is Danny',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?',
   'name?'],
  'responses': ['You can call me Danny.',
   "I'm Danny!",
   "I'm Danny aka Tech With Danny."],
  'context_set': ''},
 {'tag': 'shop'

In [94]:
try:
    with open("data.pickle","rb") as f:
        words,labels,training,output = pickle.load(f)
except:
    stemmer=LancasterStemmer()
    words=[]
    labels=[]
    docs_x=[]   # patterns
    docs_y=[]   # labels of the above patterns

    for intents in data['intents']:
        for patterns in intents['patterns']:
            word=nltk.word_tokenize(patterns)
            words.extend(word)
            docs_x.append(word)
            docs_y.append(intents['tag'])

            if intents['tag'] not in labels:
                labels.append(intents['tag'])

    words=[stemmer.stem(w.lower()) for w in words if w != '?']
    words=sorted(list(set(words)))

    labels = sorted(labels)
    #bag of words -> one hot encoding 
    # how to use one hto encoding in this case ????
    training =[]
    output=[]

    out_empty=[0 for _ in range(len(labels))]

    for x,doc in enumerate(docs_x):
        bag=[]

        wrds= [stemmer.stem(w) for w in doc if w != '?']
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row=out_empty[:]
        output_row[labels.index(docs_y[x])]=1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    # training=training.reshape(1,training.shape[0])
    output = np.array(output)

    with open("data.pickle","wb") as f:
        pickle.dump((words,labels,training,output),f)

In [95]:
epochs=1000
batch_size=8
input_shape=[len(training[0])]
num_of_labels=len(output[0])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=input_shape),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(num_of_labels, activation="softmax"),
])
try:
    model=tf.keras.models.load_model('model')
except:
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    history=model.fit(training, output, epochs=epochs, batch_size=batch_size)
    model.save('model')

In [96]:
# acc = history.history['acc']

# loss = history.history['loss']


# epochs = range(len(acc))
# plt.figure(figsize=(16,10))
# plt.plot(epochs, acc, 'b', label='Training accuracy')
# plt.plot(epochs, loss, 'r', label='Training Loss')
# plt.title('Training accuracy')
# plt.legend(loc=0)
# plt.figure()
# plt.show()

In [97]:
## bag of words function
def bag_of_words (s,words):
    bag=[0 for _ in range(len(words))]

    s_words= nltk.word_tokenize(s)
    stemmer=LancasterStemmer()
    s_words = [stemmer.stem(words.lower()) for words in s_words]

    for se in s_words:
        for i,w in enumerate (words):
            if w == se:
                bag[i]=1


    return np.array([bag])

def speak(text, language='en'):
    mp3_fo=BytesIO()
    tts =gTTS(text, lang=language)
    tts.write_to_fp(mp3_fo)
    return mp3_fo
pygame.init()
pygame.mixer.init()


def chat():
    print('Start talking with the bot! (type quit to stop)')
    while True:
        inp = input('You:')
        if inp.lower() == 'quit':
            break

        results=model.predict(bag_of_words(inp, words))
        
        results_index= np.argmax(results)
        tag=labels[results_index]
        
        for tg in data['intents']:
            if tg['tag']==tag:
                responses=tg['responses']
        out=random.choice(responses)
        print(out)
        
        sound=speak(out)
        pygame.mixer.music.load(sound, 'mp3')
        pygame.mixer.music.play()

chat()

Start talking with the bot! (type quit to stop)


C:\Users\shzso\anaconda3\envs\Chatbot\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Hello!
I am 18 years old!
I'm Danny!
You can call me Danny.
Good to see you again!


KeyboardInterrupt: Interrupted by user

In [104]:
r=sr.Recognizer()
with sr.Microphone() as source:
    print('speak bitch')
    audio = r.listen(source)
    print('time over bitch!!!!')
r.recognize_google(audio)

speak bitch
time over bitch!!!!


"how's the weather today"

In [101]:
r.recognize_google(audio)

'what the f*** b****'